In [1]:
import numpy as np
import pandas as pd
import re
import json
import os
import itertools as it
import pickle
import multiprocessing as mpr
import datetime as dt
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from pprint import pprint

import feedparser
import urllib
from bs4 import BeautifulSoup as bs

import nltk
import konlpy as knlp
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Hannanum, Kkma, Twitter

In [2]:
def rss_crawler(url, press_key):
    parsed_dict = feedparser.parse(url)
    print(parsed_dict.feed['title'])
    links = [{'press': press_key.split('_')[0],
              'title': entry.title,
              'link': entry.link,
              'datetime': entry.published} for entry in parsed_dict['entries']]
    return links

In [3]:
def news_scrapper(url, source):
    
    src_dict = {'chosun': None,
                'hani': None,
                'mk': None,
                'hk': None}

    try:
        page = urllib.request.urlopen(url)
        soup = bs(page, "html5lib")

        if source == 'chosun':
            raw_html = str(soup.find_all('div', attrs={'par'}))[1:-1]
            
            if raw_html != '[]':

                # Extraction
                pattern = re.compile(r'<[^<>]*>')
                extracted = ''.join(re.split(pattern, raw_html))

                # Substitution
                pattern = re.compile(r'^\[|google[^;]+;|\n|\t|$]')
                substituted = re.sub(pattern, '', extracted)

                # End of Article
                sliced = substituted

            elif raw_html == '[]':
                raw_html = str(soup.find_all('div', attrs='article'))[1:-1]

                # Extraction
                pattern = re.compile(r'<[^<>]*>')
                extracted = ''.join(re.split(pattern, raw_html))

                # Substitution
                pattern = re.compile(r'^\[|google[^;]+;|\n|\t|$]')
                substituted = re.sub(pattern, '', extracted)

                # End of Article
                pattern = re.compile(r'^\s*\S+ : \d+\.\d+\.\d+ \d+:\d+ \s* (.+)]')
                sliced = ''.join(re.findall(pattern, substituted))

        elif source == 'hani':
            raw_html = str(soup.find_all('div', attrs='text'))[1:-1]

            # Extraction
            pattern = re.compile(r'<[^<>]*>')
            extracted = ''.join(re.split(pattern, raw_html))

            # Substitution
            pattern = re.compile(r'^\[|google[^;]+;|\n|\t|$]')
            substituted = re.sub(pattern, '', extracted)

            # End of Article
            pattern = re.compile(r"^[\s,]*(.+[.,']+) .+ [\S]*기자 [\w]+@hani\.co\.kr")
            sliced = ''.join(re.findall(pattern, substituted))
            
        elif source == 'mk':
            raw_html = str(soup.find_all('div', attrs='art_txt'))

            # Extraction
            pattern = re.compile(r'<[^<>]*>')
            extracted = ''.join(re.split(pattern, raw_html))

            # Substitution
            pattern = re.compile(r'^\[|google[^;]+;|\n|\t|$\]')
            substituted = re.sub(pattern, '', extracted)

            # End of Article
            pattern = re.compile(r'^[\s,]*(.+)\[.+\]\[ⓒ 매일경제 &amp; mk.co.kr, 무단전재 및 재배포 금지\]')
            sliced = ''.join(re.findall(pattern, substituted))

        elif source == 'hk':
            #raw_html = str(soup.find_all('div', attrs={'news_article', 'wrap_article'}))[1:-1]
            raw_html = str(soup.find_all('div', attrs={'wrap_article'}))[1:-1]

            if raw_html == '[]':
                raw_html = str(soup.find_all('div', attrs={'news_article'}))[1:-1]

            # Extraction
            pattern = re.compile(r'<[^<>]*>')
            extracted = ''.join(re.split(pattern, raw_html))

            # Substitution
            pattern = re.compile(r'^\[|google[^;]+;|\n|\t|$]')
            substituted = re.sub(pattern, '', extracted)

            # End of Article
            pattern = re.compile(r"(.+[\.\']+).+ \S+ [\w]+@hankyung\.com")
            sliced = ''.join(re.findall(pattern, substituted))

    except urllib.error.HTTPError as e:
        print(url, str(e))
        sliced = ''
    
    print('%s (%s)' % (source, url))
    
    return sliced

In [4]:
def scrap_from_rss(feed_list):
    res = [dict(feed, **{'body': news_scrapper(feed['link'], feed['press'])})\
           for feed in feed_list]
    return res

In [5]:
def rss_dumper(rss_dict, dirname):

    # Get the RSS Feeds
    print('Listing...')
    feed_list = [rss_crawler(rss_url, press) for press, rss_url in rss_dict.items()]
    
    # Print Lengths of each Feeds
    pprint({press: len(feed) for press, feed in zip(rss_dict, feed_list)})

    print('Scrapping...')
    # Get the Full article of the Feeds
    news_list = [dict(feed, **{'body': news_scrapper(feed['link'], feed['press'])}) for feed in sum(feed_list, [])]
    print('\rScrapping...Done')

    os.makedirs(dirname, exist_ok=True)
    date_str = dt.datetime.now().strftime('%Y%m%d')
    filename = '/'.join([dirname, date_str])
    print("Dumping to '%s.*' ..." % filename)
    
    # Dump the result
    with open('%s.json' % filename, "w") as json_file:
        json_file.write(json.dumps(news_list))

    data = pd.DataFrame.from_dict(news_list)
    full_txt = '\n'.join(data['body'])
    with open('%s.txt' % filename, 'w') as text_file:
        text_file.write(full_txt)
    
    full_body_list = data['body'].tolist()
    with open('%s.dump' % filename, 'wb') as byte_file:
        pickle.dump(full_body_list, byte_file)

    print("Dumping to '%s /' ...Done" % filename)

    return news_list

In [6]:
def rss_dumper(rss_dict, dirname, worker=1):

    # Get the RSS Feeds
    print('Listing...')
    feed_list = [rss_crawler(rss_url, press) for press, rss_url in rss_dict.items()]
    
    # Print Lengths of each Feeds
    pprint({press: len(feed) for press, feed in zip(rss_dict, feed_list)})

    print('Scrapping...')
    # Get the Full article of the Feeds
    pool = mpr.pool.Pool(processes=worker)
    response = pool.map(scrap_from_rss, feed_list)
    news_list = sum(response, [])
    
    print('\rScrapping...Done')

    os.makedirs(dirname, exist_ok=True)
    date_str = dt.datetime.now().strftime('%Y%m%d')
    filename = '/'.join([dirname, date_str])
    print("Dumping to '%s.*' ..." % filename)
    
    # Dump the result
    with open('%s.json' % filename, "w") as json_file:
        json_file.write(json.dumps(news_list))

    data = pd.DataFrame.from_dict(news_list)
    full_txt = '\n'.join(data['body'])
    with open('%s.txt' % filename, 'w') as text_file:
        text_file.write(full_txt)
    
    full_body_list = data['body'].tolist()
    with open('%s.dump' % filename, 'wb') as byte_file:
        pickle.dump(full_body_list, byte_file)

    print("Dumping to '%s /' ...Done" % filename)

    return news_list

In [7]:
press_rss_dict = {'chosun_politics': 'http://www.chosun.com/site/data/rss/politics.xml',
                  'chosun_national': 'http://www.chosun.com/site/data/rss/national.xml',
                  'chosun_international': 'http://www.chosun.com/site/data/rss/international.xml',
                  'chosun_culture': 'http://www.chosun.com/site/data/rss/culture.xml',
                  'hani_politics': 'http://www.hani.co.kr/rss/politics/',
                  'hani_economy': 'http://www.hani.co.kr/rss/economy/',
                  'hani_society': 'http://www.hani.co.kr/rss/society/',
                  'hani_international': 'http://www.hani.co.kr/rss/international/',
                  'hani_culture': 'http://www.hani.co.kr/rss/culture/',
                  'hani_science': 'http://www.hani.co.kr/rss/science/',
                  'mk_economy': 'http://file.mk.co.kr/news/rss/rss_30100041.xml',
                  'mk_politics': 'http://file.mk.co.kr/news/rss/rss_30200030.xml',
                  'mk_society': 'http://file.mk.co.kr/news/rss/rss_50400012.xml',
                  'mk_international': 'http://file.mk.co.kr/news/rss/rss_30300018.xml',
                  'mk_corp': 'http://file.mk.co.kr/news/rss/rss_50100032.xml',
                  'mk_stock': 'http://file.mk.co.kr/news/rss/rss_50200011.xml',
                  'mk_realestate': 'http://file.mk.co.kr/news/rss/rss_50300009.xml',
                  'mk_culture': 'http://file.mk.co.kr/news/rss/rss_30000023.xml',
                  'hk_stock': 'http://rss.hankyung.com/new/news_stock.xml',
                  'hk_economy': 'http://rss.hankyung.com/new/news_economy.xml',
                  'hk_industry': 'http://rss.hankyung.com/new/news_industry.xml',
                  'hk_realestate': 'http://rss.hankyung.com/new/news_estate.xml',
                  'hk_politics': 'http://rss.hankyung.com/new/news_politics.xml',
                  'hk_society': 'http://rss.hankyung.com/new/news_society.xml',
                 }

In [8]:
if __name__ == '__main__':
    rss_dumper(press_rss_dict, 'news_20171129', worker=8)

Listing...
조선닷컴 : 정치
조선닷컴 : 사회
조선닷컴 : 국제
조선닷컴 : 문화
전체기사 : 정치 :  뉴스 : 한겨레  뉴스 - 인터넷한겨레
전체기사 : 경제 :  뉴스 : 한겨레  뉴스 - 인터넷한겨레
전체기사 : 사회 :  뉴스 : 한겨레  뉴스 - 인터넷한겨레
전체기사 : 국제 :  뉴스 : 한겨레  뉴스 - 인터넷한겨레
전체기사 : 문화 :  뉴스 : 한겨레  뉴스 - 인터넷한겨레
전체기사 : 과학 :  뉴스 : 한겨레  뉴스 - 인터넷한겨레
MK:경제·금융 신문기사
MK:정치 신문기사
MK:사회 신문기사
MK:국제 신문기사
MK:기업·경영 신문기사
MK:증권 신문기사
MK:부동산 신문기사
MK:문화·연예 주요기사
한국경제 | 뉴스 | 증권
한국경제 | 뉴스 | 경제/금융
한국경제 | 뉴스 | 산업/IT
한국경제 | 뉴스 | 부동산
한국경제 | 뉴스 | 정치
한국경제 | 뉴스 | 사회
{'chosun_culture': 30,
 'chosun_international': 30,
 'chosun_national': 30,
 'chosun_politics': 30,
 'hani_culture': 30,
 'hani_economy': 30,
 'hani_international': 30,
 'hani_politics': 30,
 'hani_science': 30,
 'hani_society': 30,
 'hk_economy': 20,
 'hk_industry': 20,
 'hk_politics': 1,
 'hk_realestate': 20,
 'hk_society': 20,
 'hk_stock': 20,
 'mk_corp': 50,
 'mk_culture': 50,
 'mk_economy': 50,
 'mk_international': 50,
 'mk_politics': 50,
 'mk_realestate': 50,
 'mk_society': 50,
 'mk_stock': 50}
Scrapping...
chosun (http://news.chosu

chosun (http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900493.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/27/2017112701463.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900053.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900481.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/27/2017112700163.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/28/2017112801925.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900051.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900343.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900059.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/28/2017112801921.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/27/2017112700152.html)
chosun (http://news.chosun.com/site/data/html_dir/2017/11/29/2017112900341.html)
chosun (http://news.chosun.c

mk (http://news.mk.co.kr/newsRead.php?no=789094&year=2017)
hani (http://www.hani.co.kr/arti/international/japan/821046.html)
hani (http://www.hani.co.kr/arti/politics/defense/821226.html)
mk (http://news.mk.co.kr/newsRead.php?no=789093&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=790202&year=2017)
hani (http://www.hani.co.kr/arti/science//820308.html)
mk (http://news.mk.co.kr/newsRead.php?no=789068&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=789067&year=2017)
hani (http://www.hani.co.kr/arti/economy/economy_general/821199.html)
mk (http://news.mk.co.kr/newsRead.php?no=789066&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=790194&year=2017)
hani (http://www.hani.co.kr/arti/culture/movie/821048.html)
hani (http://www.hani.co.kr/arti/society/society_general/821221.html)
mk (http://news.mk.co.kr/newsRead.php?no=789065&year=2017)
hani (http://www.hani.co.kr/arti/international/china/821039.html)
hani (http://www.hani.co.kr/arti/politics/assembly/821219.html)
mk (http://new

mk (http://news.mk.co.kr/newsRead.php?no=789522&year=2017)
hani (http://www.hani.co.kr/arti/politics/assembly/821192.html)
mk (http://news.mk.co.kr/newsRead.php?no=788085&year=2017)
hani (http://www.hani.co.kr/arti/society/society_general/821172.html)
mk (http://news.mk.co.kr/newsRead.php?no=789510&year=2017)
hani (http://www.hani.co.kr/arti/culture/movie/820806.html)
hani (http://www.hani.co.kr/arti/international/america/820877.html)
mk (http://news.mk.co.kr/newsRead.php?no=789452&year=2017)
hani (http://www.hani.co.kr/arti/economy/economy_general/821135.html)
hani (http://www.hani.co.kr/arti/science/science_general/818936.html)
mk (http://news.mk.co.kr/newsRead.php?no=788056&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=789430&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=789408&year=2017)
hani (http://www.hani.co.kr/arti/society/society_general/821170.html)
hani (http://www.hani.co.kr/arti/politics/assembly/821189.html)
mk (http://news.mk.co.kr/newsRead.php?no=789407&yea

hani (http://www.hani.co.kr/arti/culture/music/820608.html)
mk (http://news.mk.co.kr/newsRead.php?no=791573&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=790506&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=791509&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=791239&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=790466&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=791505&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=789973&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=791237&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=790445&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=791512&year=2017)
hk (http://www.hankyung.com/news/app/newsview.php?aid=2017112917976&sid=newsStock&nid=000&rss=r)
hani (http://www.hani.co.kr/arti/science/science_general/817593.html)
mk (http://news.mk.co.kr/newsRead.php?no=790835&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=791443&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=790224&year=2017)
mk (ht

hk (http://www.hankyung.com/news/app/newsview.php?aid=2017112996996&sid=newsEconomy&nid=000&rss=r)
mk (http://news.mk.co.kr/newsRead.php?no=786812&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=786439&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=789049&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=786208&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=786204&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=789048&year=2017)
hk (http://www.hankyung.com/news/app/newsview.php?aid=2017112907186&sid=newsEconomy&nid=000&rss=r)
mk (http://news.mk.co.kr/newsRead.php?no=786191&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=791146&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=786104&year=2017)
hk (http://www.hankyung.com/news/app/newsview.php?aid=2017112908276&sid=newsStock&nid=000&rss=r)
mk (http://news.mk.co.kr/newsRead.php?no=789017&year=2017)
hk (http://www.hankyung.com/news/app/newsview.php?aid=201711286773g&sid=newsIndustry&nid=000&rss=r)
mk (http://news

mk (http://news.mk.co.kr/newsRead.php?no=783500&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=787229&year=2017)
hk (http://www.hankyung.com/news/app/newsview.php?aid=2017112998786&sid=newsEconomy&nid=000&rss=r)
mk (http://news.mk.co.kr/newsRead.php?no=790659&year=2017)
hk (http://www.hankyung.com/news/app/newsview.php?aid=201711290615g&sid=newsEstate&nid=000&rss=r)
mk (http://news.mk.co.kr/newsRead.php?no=787164&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=790599&year=2017)
http://www.hankyung.com/news/app/newsview.php?aid=&sid=newsPolitics&nid=000&rss=r HTTP Error 404: Not Found
hk (http://www.hankyung.com/news/app/newsview.php?aid=&sid=newsPolitics&nid=000&rss=r)
mk (http://news.mk.co.kr/newsRead.php?no=790447&year=2017)
hk (http://www.hankyung.com/news/app/newsview.php?aid=2017112904576&sid=newsStock&nid=000&rss=r)
mk (http://news.mk.co.kr/newsRead.php?no=790567&year=2017)
mk (http://news.mk.co.kr/newsRead.php?no=790419&year=2017)
hk (http://www.hankyung.com/news/app/ne

# Load Data

## DataFrame from `json`

In [13]:
fname = 'news_20171129/20171129.json'
data = pd.read_json(fname)
data['datetime'] = pd.to_datetime(data['datetime'])
data

,body,datetime,link,press,title
0,바른정당 유승민 대표는 29일 북한이 새벽 동해상으로 장거리 탄도미사일 1발을 기...,2017-11-29 06:06:59,http://news.chosun.com/site/data/html_dir/2017...,chosun,"유승민, 北미사일 발사에 ""文정부 너무 한가로워…대북대책 전면 수정해야"""
1,29일 북한이 75일 만에 미사일 발사 도발을 한 데 대한 정부의 대응이 상당히...,2017-11-29 05:42:14,http://news.chosun.com/site/data/html_dir/2017...,chosun,"北 미사일 발사, 어떻게 사전에 파악해 신속 대응할 수 있었나"
2,이낙연 국무총리는 부정청탁 및 금품 등 수수의 금지에 관한 법률(청탁금지법)에서 ...,2017-11-29 05:36:36,http://news.chosun.com/site/data/html_dir/2017...,chosun,"李총리 ""'김영란법' 선물비 상향, 수정안 준비中…내년 설 이전 개정"""
3,자유한국당 소속인 심재철 국회부의장은 29일 ‘문재인 대통령과 임종석 비서실장 등...,2017-11-29 04:27:11,http://news.chosun.com/site/data/html_dir/2017...,chosun,"심재철, 與반발에도 ""文정부 6개월 '이념적 폭동'…내란죄 성립"""
4,"국가정보원이 29일 대공 수사권을 타기관에 이관하고, 국가보안법상 찬양고무죄·불고...",2017-11-29 04:13:15,http://news.chosun.com/site/data/html_dir/2017...,chosun,"국정원 ""모든 수사권 이관·폐지…찬양고무죄 관련 정보수집도 않기로"""
5,"국가정보원은 29일 북한이 ICBM급 미사일을 발사한 것과 관련, “예견된 도발이...",2017-11-29 03:56:00,http://news.chosun.com/site/data/html_dir/2017...,chosun,"국정원 ""北, 추가 핵실험 가능성…이번 미사일은 가장 진전된 것"""
6,"국회 정보위는 29일 전체회의를 열고, 국가정보원의 내년도 예산안에서 순수 특수활...",2017-11-29 03:52:11,http://news.chosun.com/site/data/html_dir/2017...,chosun,"국회 정보위 ""내년 예산안서 국정원 특활비 680억원 감액"""
7,국가정보원은 29일 북한이 ICBM(대륙 간 탄도 미사일)급 미사일을 발사한 것과...,2017-11-29 03:50:15,http://news.chosun.com/site/data/html_dir/2017...,chosun,"국정원 ""北미사일, 美타격 능력 과시…中 대북제재에 불만 표출"""
8,북한이 29일 신형 대륙간탄도미사일(ICBM) 화성-15형의 발사에 성공했다고 발...,2017-11-29 03:35:27,http://news.chosun.com/site/data/html_dir/2017...,chosun,"北 ""ICBM 화성-15형 시험발사 성공""…국가핵무력 완성 주장"
9,자유한국당 홍준표 대표가 29일 한병도 청와대 신임 정무수석을 만나 “우리 당 의...,2017-11-29 02:53:38,http://news.chosun.com/site/data/html_dir/2017...,chosun,"홍준표, 신임 靑정무수석에 ""우리 의원들 자꾸 잡아가지 마"""


## `text`

In [14]:
fname = 'news_20171129/20171129.txt'
with open(fname, 'r') as f:
    full_txt = f.read()
full_txt

' 바른정당 유승민 대표는 29일 북한이 새벽 동해상으로 장거리 탄도미사일 1발을 기습 발사한 것에 대해 “지난 6개월 동안 해왔던 (대북) 대책을 전면수정하고 재검토해야 한다”고 했다. 유 대표는 이날 바른정당 최고위원·원외위원장 연석회의에서 “지금부터라도 핵과 미사일 대책에 대해 완전 새로운 궤도로 들어가야 한다”고 했다. 유 대표는 “시간은 가는데 북한의 핵과 미사일은 완성과 실전 배치에 이르고 있다”며 “우리 정부와 미국 당사자가 너무 안이하고 한가로운 것이 아닌가”라고 했다. 이어 “지금 문재인 정부는 북한과 대화를 입 밖에 꺼낼 때가 아니다”라며 “이제까지 시간 보내기와 같은 안이한 대응을 해 이런 사태를 불러왔으니 이를 인정하고 반성해야 한다”고 했다. 한편 유 대표는 전술핵 재배치와 관련해선 “미국이 반대하더라도 전술핵 재배치, 핵 공유 카드는 반드시 가지고 있으면서 대응해야 효과가 있다”며 “군사적 대비태세도 강화해 북한 이동식 발사대를 추적하는 정찰 자산인 조인트 스타즈(J-STARS)를 도입해야 한다”고 했다. \n  29일 북한이 75일 만에 미사일 발사 도발을 한 데 대한 정부의 대응이 상당히 빨랐다는 평가가 나오고 있다. 합참은 이번 미사일 발사 지역에서 지금까지 북한이 미사일을 발사한 적이 없다고 밝혔다. 이런 상황에서도 미사일 발사 2분 만에 대통령 보고가 이뤄지고, 6분 만에 발사 원점 타격 훈련이 실시됐다. 전문가들은 “한·미·일 3국 공조하에 북한 움직임을 미리 감지했기 때문에 신속 대응할 수 있었다”며 “실전 대응 능력을 높이면서 북한에게도 경고를 주는 계기가 됐다”고 말한다. 그렇다면 한·미·일 당국은 어떻게 북한의 도발 징후를 미리 알 수 있었을까. 북한은 이날 새벽 3시17분 평양에서 북쪽으로 30㎞ 떨어진 평안남도 평성 일대에서 ICBM(대륙간탄도미사일)급 장거리 미사일을 발사했다. 우리 합참은 새벽 3시18분쯤 E-737(피스아이)가 처음 탐지했고, 이후 동해에서 작전 중인 이지스함과 조기경보레이더에서도 포착했다고 

## `byte` Object

In [15]:
fname = 'news_20171129/20171129.dump'
with open(fname, 'rb') as f:
    full_list = pickle.load(f)
    
full_list

[' 바른정당 유승민 대표는 29일 북한이 새벽 동해상으로 장거리 탄도미사일 1발을 기습 발사한 것에 대해 “지난 6개월 동안 해왔던 (대북) 대책을 전면수정하고 재검토해야 한다”고 했다. 유 대표는 이날 바른정당 최고위원·원외위원장 연석회의에서 “지금부터라도 핵과 미사일 대책에 대해 완전 새로운 궤도로 들어가야 한다”고 했다. 유 대표는 “시간은 가는데 북한의 핵과 미사일은 완성과 실전 배치에 이르고 있다”며 “우리 정부와 미국 당사자가 너무 안이하고 한가로운 것이 아닌가”라고 했다. 이어 “지금 문재인 정부는 북한과 대화를 입 밖에 꺼낼 때가 아니다”라며 “이제까지 시간 보내기와 같은 안이한 대응을 해 이런 사태를 불러왔으니 이를 인정하고 반성해야 한다”고 했다. 한편 유 대표는 전술핵 재배치와 관련해선 “미국이 반대하더라도 전술핵 재배치, 핵 공유 카드는 반드시 가지고 있으면서 대응해야 효과가 있다”며 “군사적 대비태세도 강화해 북한 이동식 발사대를 추적하는 정찰 자산인 조인트 스타즈(J-STARS)를 도입해야 한다”고 했다. ',
 '  29일 북한이 75일 만에 미사일 발사 도발을 한 데 대한 정부의 대응이 상당히 빨랐다는 평가가 나오고 있다. 합참은 이번 미사일 발사 지역에서 지금까지 북한이 미사일을 발사한 적이 없다고 밝혔다. 이런 상황에서도 미사일 발사 2분 만에 대통령 보고가 이뤄지고, 6분 만에 발사 원점 타격 훈련이 실시됐다. 전문가들은 “한·미·일 3국 공조하에 북한 움직임을 미리 감지했기 때문에 신속 대응할 수 있었다”며 “실전 대응 능력을 높이면서 북한에게도 경고를 주는 계기가 됐다”고 말한다. 그렇다면 한·미·일 당국은 어떻게 북한의 도발 징후를 미리 알 수 있었을까. 북한은 이날 새벽 3시17분 평양에서 북쪽으로 30㎞ 떨어진 평안남도 평성 일대에서 ICBM(대륙간탄도미사일)급 장거리 미사일을 발사했다. 우리 합참은 새벽 3시18분쯤 E-737(피스아이)가 처음 탐지했고, 이후 동해에서 작전 중인 이지스함과 조기경보레이더에서도 포착

Done.